In [31]:
import numpy as np
import scipy as sp
import math
import matplotlib.pyplot as plt
from scipy.special import genlaguerre as L
from scipy.integrate import solve_ivp as solve
from scipy.integrate import odeint as odeint
from scipy.integrate import complex_ode as ode
import time
from numba import jit

In [13]:
def prob(n,nbar):
    return 1.0/(nbar+1)*(nbar/(nbar+1))**n    #returns prob of being in state n given nbar

def groundRho(Ncut, nbar):
    rho = np.array([0.0+0.0j]*3*(Ncut+1))
    for n in range(Ncut):
        rho[3*n] = prob(n,nbar)
    return rho

def nbar(rho):
    Ncut = rho.shape[0]//3
    nbar = 0.0
    for n in range(Ncut):
        nbar += n*(rho[n*3].real + rho[n*3+1].real)
    return nbar

def rhogg(rho):
    rhogg = 0.0
    Ncut = rho.shape[0]//3
    for n in range(Ncut):
        rhogg += rho[(3*n)].real
    return rhogg

def rhoee(rho):
    rhoee = 0.0
    Ncut = rho.shape[0]//3
    for n in range(Ncut):
        rhoee += rho[(3*n)+1].real
    return rhoee

In [11]:
def heatEqns(t, rho, nbardot):
    Ncut = rho.shape[0]//3 - 1
    rhoDot = [0.0+0.0j]*((Ncut+1)*3)
    for ii in range(3):
        rhoDot[0+ii] = nbardot*(-rho[0+ii]+rho[3*1+ii])
        rhoDot[3*Ncut+ii] = nbardot*(-Ncut*rho[3*Ncut+ii]+Ncut*rho[3*(Ncut-1)+ii])
        #Note*** the above line is true in the limit that rho[Ncut+1,ii]=rho[Ncut,ii] (fair assumption for large Ncut I think)
        for n in range(1,Ncut-1):
            rhoDot[3*n+ii] = nbardot*(-(2.0*n+1.0)*rho[3*n+ii] + (n+1.0)*rho[3*(n+1)+ii] + n*rho[3*(n-1)+ii])
    return rhoDot

def heat(rho0, t, nbardot):
    rho = solve(heatEqns, [0.0,t], rho0, args=[nbardot]).y[:,-1]
    return rho

def fastHeat(rho0, t, nbardot, Nstop):
    #Nstop = 100
    Ncut = rho0.shape[0]//3 - 1
    rho = np.array([0.0+0.0j]*3*(Ncut+1))
    rho[:3*Nstop] = solve(heatEqns, [0.0,t], rho0[:3*Nstop], args=[nbardot]).y[:,-1]
    rho[3*Nstop:] = rho0[3*Nstop:]
    return rho

In [42]:
def heatEqns(t, rho, nbardot):
    Ncut = rho.shape[0]//3 - 1
    rhoDot = [0.0+0.0j]*((Ncut+1)*3)
    for ii in range(3):
        rhoDot[0+ii] = nbardot*(-rho[0+ii]+rho[3*1+ii])
        rhoDot[3*Ncut+ii] = nbardot*(-Ncut*rho[3*Ncut+ii]+Ncut*rho[3*(Ncut-1)+ii])
        #Note*** the above line is true in the limit that rho[Ncut+1,ii]=rho[Ncut,ii] (fair assumption for large Ncut I think)
        for n in range(1,Ncut-1):
            rhoDot[3*n+ii] = nbardot*(-(2.0*n+1.0)*rho[3*n+ii] + (n+1.0)*rho[3*(n+1)+ii] + n*rho[3*(n-1)+ii])
    return rhoDot

def heatb(rho0, t, nbardot):
    ts = np.linspace(0.0,t,10)
    rho = ode(heatEqns, rho0, ts, (nbardot,)).y[:,-1]
    return rho

def fastHeatb(rho0, t, nbardot, Nstop):
    #Nstop = 100
    Ncut = rho0.shape[0]//3 - 1
    rho = np.array([0.0+0.0j]*3*(Ncut+1))
    rho[:3*Nstop] = solve(heatEqns, [0.0,t], rho0[:3*Nstop], t_eval=[t], args=[nbardot]).y[:,-1]
    rho[3*Nstop:] = rho0[3*Nstop:]
    return rho

In [44]:
rho0 = groundRho(1000, 0.0)
start = time.time()
Nbar = nbar(fastHeatb(rho0, 1.0, 1.0, 1000))
end = time.time()
print(end-start)

start = time.time()
Nbar = nbar(fastHeat(rho0, 1.0, 1.0, 1000))
end = time.time()
print(end-start)

1.0368945598602295
1.031006097793579


In [34]:
heatb(rho0, 1.0, 1.0)

TypeError: complex_ode.__init__() takes from 2 to 3 positional arguments but 5 were given

In [39]:
([1.0])

[1.0]

In [41]:
np.array([1.0])

array([1.])